### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline


# Read in the MovieTweetings dataset originally taken from https://github.com/sidooms/MovieTweetings/tree/master/latest
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
# cell for work
display(movies.head())
display(reviews.head())


,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [3]:
display(movies.shape)
display(reviews.shape)


(35479, 3)

(863866, 4)

In [4]:
display(movies.info())
display(reviews.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35479 entries, 0 to 35478
Data columns (total 3 columns):
movie_id    35479 non-null object
movie       35479 non-null object
genre       35230 non-null object
dtypes: object(3)
memory usage: 831.6+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 863866 entries, 0 to 863865
Data columns (total 4 columns):
user_id      863866 non-null object
movie_id     863866 non-null object
rating       863866 non-null int64
timestamp    863866 non-null object
dtypes: int64(1), object(3)
memory usage: 26.4+ MB


None

In [5]:
movies.movie_id.nunique()

35479

In [6]:
# Use our findings to match each variable to the correct statement in the dictionary

genre_list = []
for i in movies.genre:
    try:
        genre_list.extend(i.split('|'))
    except AttributeError:
        pass
    
genre_list = (list(set(genre_list)))


dict_sol1 = {
'The number of movies in the dataset': movies.shape[0], 
'The number of ratings in the dataset': reviews.shape[0],
'The number of different genres' : len(genre_list), 
'The number of unique users in the dataset': reviews.user_id.nunique(), 
'The number missing ratings in the reviews dataset': reviews.rating.isnull().sum(), 
'The average rating given across all ratings': np.round(reviews.rating.mean(), 0),
'The minimum rating given across all ratings': reviews.rating.min(),
'The maximum rating given across all ratings': reviews.rating.max()
}

dict_sol1

{'The number of movies in the dataset': 35479,
 'The number of ratings in the dataset': 863866,
 'The number of different genres': 28,
 'The number of unique users in the dataset': 67353,
 'The number missing ratings in the reviews dataset': 0,
 'The average rating given across all ratings': 7.0,
 'The minimum rating given across all ratings': 0,
 'The maximum rating given across all ratings': 10}

#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [7]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [8]:
print(movies.loc[3]['movie'])
movies['date'] = pd.to_numeric(movies['movie'].apply(lambda x:x[-5:-1] if x[-1]==')' else np.nan))
movies.sample(10)
movies.info()

The Oxford and Cambridge University Boat Race (1895)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35479 entries, 0 to 35478
Data columns (total 4 columns):
movie_id    35479 non-null object
movie       35479 non-null object
genre       35230 non-null object
date        35479 non-null int64
dtypes: int64(1), object(3)
memory usage: 1.1+ MB


In [9]:
display(movies.head())
movies.date.isnull().sum()

,movie_id,movie,genre,date
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895
4,0000091,Le manoir du diable (1896),Short|Horror,1896


0

genre_list = []
for i in range(movies.shape[0]):
    genre_list.append(str(movies.iloc[i]['genre']).split('|'))
import itertools
genre_flat = list(itertools.chain(*genre_list))
len(set(genre_list))


In [10]:
def dummy_year(i):
    if i<1900:
        return '1800s'
    elif 1899<i<2000:
        return '1900s'
    else:
        return '2000s'
    
    
movies['yr'] = movies['date'].apply(dummy_year)
movies.sample(5)

,movie_id,movie,genre,date,yr
29539,4458206,Kod Adi K.O.Z. (2015),Crime|Mystery,2015,2000s
20696,1789885,Pus (2010),Drama,2010,2000s
34862,9095324,Coldplay: A Head Full of Dreams (2018),Documentary|Music,2018,2000s
14684,0467596,Kasal? (1980),Drama|Romance,1980,1900s
22353,2108546,Jackie (2012),Comedy|Drama,2012,2000s


In [11]:
movies = pd.get_dummies(movies,columns=['yr'],prefix='',prefix_sep='')
movies.head()


,movie_id,movie,genre,date,1800s,1900s,2000s
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0


In [12]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [13]:
import datetime

change_timestamp = lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')

reviews['date'] = reviews['timestamp'].apply(change_timestamp)

In [14]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0114508,8,1381006850,2013-10-05 17:00:50
1,2,0208092,5,1586466072,2020-04-09 17:01:12
2,2,0358273,9,1579057827,2020-01-14 22:10:27
3,2,10039344,5,1578603053,2020-01-09 15:50:53
4,2,6751668,9,1578955697,2020-01-13 17:48:17


In [15]:
genre_list

['Sci-Fi',
 'Documentary',
 'Crime',
 'Biography',
 'Horror',
 'Talk-Show',
 'Action',
 'Music',
 'Game-Show',
 'Comedy',
 'News',
 'Romance',
 'Mystery',
 'Thriller',
 'Reality-TV',
 'Adventure',
 'Film-Noir',
 'Drama',
 'History',
 'Sport',
 'Animation',
 'Adult',
 'Family',
 'War',
 'Western',
 'Short',
 'Fantasy',
 'Musical']

In [16]:
movies.head()

,movie_id,movie,genre,date,1800s,1900s,2000s
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0


In [17]:
movies2 = movies.copy()
#movies2[0] = np.zeros(35479)

In [18]:
for i in range (len(genre_list)):
    movies2[i] = np.zeros(35479)

In [19]:
movies2.shape

(35479, 35)

In [20]:
movies3 =  movies2.copy()

In [21]:
a = {}
for  i  in range(len(genre_list)):
    a[i]=genre_list[i]  
    
movies3.rename(columns=a, inplace=True)
movies3.head()

,movie_id,movie,genre,date,1800s,1900s,2000s,Sci-Fi,Documentary,Crime,...,History,Sport,Animation,Adult,Family,War,Western,Short,Fantasy,Musical
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
movies4 = movies3.copy()
movies4.genre[5]

'Short|Comedy|Horror'

In [23]:
for i in genre_list:
    for j in range(movies4.shape[0]):
        if i in list(str(movies4.genre[j]).split('|')):
            movies4.loc[j,i]=1

In [24]:
list(movies4.genre[5].split('|'))
movies4.loc[5,'Short']=1

In [25]:
movies4.head(6).style

,movie_id,movie,genre,date,1800s,1900s,2000s,Sci-Fi,Documentary,Crime,Biography,Horror,Talk-Show,Action,Music,Game-Show,Comedy,News,Romance,Mystery,Thriller,Reality-TV,Adventure,Film-Noir,Drama,History,Sport,Animation,Adult,Family,War,Western,Short,Fantasy,Musical
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race (1895),nan,1895,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5,0000131,Une nuit terrible (1896),Short|Comedy|Horror,1896,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
